# Training MNIST Model

The goal of this is to create a larger-than-necessary model for classifying handwritten MNIST digits. After training, the model will be serialized in a binary format to later be run using **raw** CUDA and OpenCL.

In [1]:
# required imports
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
# Define the neural network model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10) # 0 - 9 digits
    
    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = SimpleNN()

In [3]:
# Hyperparameters
batch_size = 64
learning_rate = 0.001
epochs = 5
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [4]:
# data loading
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Datasets and DataLoaders
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Failed to download (trying next):
HTTP Error 403: Forbidden




  0%|                                                                                                                                                                                                                                                                                | 0/9912422 [00:00<?, ?it/s]


  2%|████▏                                                                                                                                                                                                                                                         | 163840/9912422 [00:00<00:06, 1552624.63it/s]


 14%|██████████████████████████████████▎                                                                                                                                                                                                                          | 1343488/9912422 [00:00<00:01, 7251945.97it/s]


 40%|███████████████████████████████████████████████████████████████████████████████████████████████████▉                                                                                                                                                        | 3932160/9912422 [00:00<00:00, 13702651.46it/s]

...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1648877/1648877 [00:00<00:00, 5050499.51it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden




  0%|                                                                                                                                                                                                                                                                                   | 0/4542 [00:00<?, ?it/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<00:00, 12064932.72it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [5]:
# training the model
model.train()
for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}')

Epoch 1/5, Loss: 0.3925219791720925


Epoch 2/5, Loss: 0.1883651724359247


Epoch 3/5, Loss: 0.1337862066648551


Epoch 4/5, Loss: 0.11023652851124847


Epoch 5/5, Loss: 0.09296311692087485


In [6]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Accuracy of the model on the test images: {100 * correct / total}%')

Accuracy of the model on the test images: 96.21%


In [7]:
# rather sloppy section to save model weights in a binary format
# somewhat difficult for some reason

import glob
import os

os.system('rm *.bin')

rm: cannot remove '*.bin': No such file or directory


256

In [8]:
# checking the distribution of weights

# this comes after seeing the bill daily talk where weights are: normally distributed **before** pruning and non-normally distributed after pruning
i = 0
for layer in model.children():
    if isinstance(layer, nn.Linear):
        state = layer.state_dict()
        state['weight'].flatten().data.cpu().numpy().tofile(f'weights{i}.bin')
        state['bias'].flatten().data.cpu().numpy().tofile(f'bias{i}.bin')
        i += 1

In [9]:
with open('model', 'wb') as f:
    for file in glob.glob('*.bin'):
        with open(file, 'rb') as f2:
            f.write(f2.read())

os.system('rm *.bin')

0